In [303]:
import pandas as pd
import numpy as np
import ot

In [304]:
fjc = pd.read_csv('FJC_2018.csv')


/var/folders/rp/lgskb8cx1vzgzb8nb18p5vxw0000gn/T/ipykernel_43359/3443644311.py:1: DtypeWarning: Columns (4,11,42,54,65,96,108,113,120,125) have mixed types. Specify dtype option on import or set low_memory=False.
  fjc = pd.read_csv('FJC_2018.csv')


In [307]:
ussc = pd.read_csv('mod_ussc_2018.csv')

In [309]:
ussc = ussc[ussc['SENTYR'] == 2018]
ussc.reset_index(drop=True, inplace=True)

In [311]:
ussc_new = ussc[['USSCIDN','SENTMON', 'SENTYR', 'DISTRICT','SENTTOT_new', 'FINE', 'PROBATN', 'floorSENTTOT', 'roundSENTTOT','USSCcode1', 'USSCcode2', 'USSCcode3', 'USSCcode4', 'USSCcode5']]
ussc_new = ussc_new.rename({'SENTTOT_new': 'PRISTOT', 'FINE': 'FINETOT', 'PROBATN': 'PROBTOT'}, axis=1)

In [313]:
fjc.rename(columns={ fjc.columns[0]: "ID" }, inplace = True)

In [315]:
fjc_new = fjc[['ID','SENTMON', 'SENTYR', 'DISTRICT', 'PRISTOT', 'FINETOT', 'PROBTOT', 'FJDcode1', 'FJDcode2', 'FJDcode3', 'FJDcode4', 'FJDcode5', 'FJDcode6', 'FJDcode7', 'FJDcode8', 'FJDcode9', 'FJDcode10']]
fjc_new.replace(-8, 'NA', inplace=True)
fjc_new.replace('-8', 'NA', inplace=True)
fjc_new = fjc_new

/var/folders/rp/lgskb8cx1vzgzb8nb18p5vxw0000gn/T/ipykernel_43359/1762165008.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fjc_new.replace(-8, 'NA', inplace=True)
/var/folders/rp/lgskb8cx1vzgzb8nb18p5vxw0000gn/T/ipykernel_43359/1762165008.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fjc_new.replace('-8', 'NA', inplace=True)


In [316]:
pd.merge(ussc_new, fjc_new, on=['SENTMON', 'SENTYR', 'DISTRICT', 'PRISTOT', 'FINETOT', 'PROBTOT'])

,USSCIDN,SENTMON,SENTYR,DISTRICT,PRISTOT,FINETOT,PROBTOT,floorSENTTOT,roundSENTTOT,USSCcode1,...,FJDcode1,FJDcode2,FJDcode3,FJDcode4,FJDcode5,FJDcode6,FJDcode7,FJDcode8,FJDcode9,FJDcode10
0,2436847,1,2018,88,188.0,0,0,188.0,188.0,81326.0,...,21841,NaN,NaN,NaN,NaN,21841,NaN,NaN,NaN,NaN
1,2438539,1,2018,68,300.0,0,0,300.0,300.0,81326.0,...,21846,18924.0,21841.0,NaN,NaN,21846,21841.0,18924.0,NaN,NaN
2,2438539,1,2018,68,300.0,0,0,300.0,300.0,81326.0,...,18924,21841.0,18922.0,18922.0,18922.0,21841,18922.0,18922.0,18924.0,18922.0
3,2443206,1,2018,68,300.0,0,0,300.0,300.0,81326.0,...,21846,18924.0,21841.0,NaN,NaN,21846,21841.0,18924.0,NaN,NaN
4,2443206,1,2018,68,300.0,0,0,300.0,300.0,81326.0,...,18924,21841.0,18922.0,18922.0,18922.0,21841,18922.0,18922.0,18924.0,18922.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270173,2516184,7,2018,4,0.0,0,0,NaN,NaN,NaN,...,81326,NaN,NaN,NaN,NaN,81326,NaN,NaN,NaN,NaN
270174,2516184,7,2018,4,0.0,0,0,NaN,NaN,NaN,...,81326,NaN,NaN,NaN,NaN,81326,NaN,NaN,NaN,NaN
270175,2516184,7,2018,4,0.0,0,0,NaN,NaN,NaN,...,81326,NaN,NaN,NaN,NaN,81326,NaN,NaN,NaN,NaN
270176,2516184,7,2018,4,0.0,0,0,NaN,NaN,NaN,...,81326,NaN,NaN,NaN,NaN,81326,NaN,NaN,NaN,NaN


In [ ]:
#taking subset from matched cases and trying out optimal transport

In [317]:
matches = pd.read_csv('matched_2018.csv')
matches

,ID,SENTMON,SENTYR,DISTRICT,PRISTOT,FINETOT,PROBTOT,FJDcode1,FJDcode2,FJDcode3,...,FJDcode9,FJDcode10,USSCIDN,floorSENTTOT,roundSENTTOT,USSCcode1,USSCcode2,USSCcode3,USSCcode4,USSCcode5
0,4366217.0,6,2018,90,216.0,0,0,18924.0,18924.0,18924.0,...,21841.0,18922.0,2472159.0,216.0,216.0,182252.0,NaN,NaN,NaN,NaN
1,4366322.0,7,2018,90,60.0,0,0,18922.0,NaN,NaN,...,NaN,NaN,2478292.0,60.0,60.0,NaN,NaN,NaN,NaN,NaN
2,4366387.0,1,2018,90,144.0,0,0,21846.0,18924.0,21841.0,...,18924.0,21841.0,2442616.0,144.0,144.0,81326.0,81326.0,NaN,NaN,NaN
3,4366406.0,4,2018,90,60.0,0,0,18924.0,18924.0,21841.0,...,18922.0,18922.0,2460526.0,60.0,60.0,181349.0,NaN,NaN,NaN,NaN
4,4366428.0,2,2018,90,47.0,0,0,21841.0,18922.0,21841.0,...,21841.0,18924.0,2446428.0,47.0,47.0,182252.0,182252.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4465,4578806.0,3,2018,34,0.0,0,0,81326.0,NaN,NaN,...,NaN,NaN,2452917.0,0.0,0.0,21846.0,18922.0,21841.0,18924.0,21841.0
4466,4578831.0,8,2018,34,0.0,0,0,81326.0,NaN,NaN,...,NaN,NaN,2481538.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4467,4578982.0,4,2018,34,0.0,0,0,81326.0,NaN,NaN,...,NaN,NaN,2457742.0,0.0,0.0,181791.0,NaN,NaN,NaN,NaN
4468,4579012.0,9,2018,34,0.0,0,0,81326.0,NaN,NaN,...,NaN,NaN,2488111.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [318]:
matches_100=matches.iloc[0:100,:] #taking a sample of 100 to work with
matches_100

,ID,SENTMON,SENTYR,DISTRICT,PRISTOT,FINETOT,PROBTOT,FJDcode1,FJDcode2,FJDcode3,...,FJDcode9,FJDcode10,USSCIDN,floorSENTTOT,roundSENTTOT,USSCcode1,USSCcode2,USSCcode3,USSCcode4,USSCcode5
0,4366217.0,6,2018,90,216.0,0,0,18924.0,18924.0,18924.0,...,21841.0,18922.0,2472159.0,216.0,216.0,182252.0,NaN,NaN,NaN,NaN
1,4366322.0,7,2018,90,60.0,0,0,18922.0,NaN,NaN,...,NaN,NaN,2478292.0,60.0,60.0,NaN,NaN,NaN,NaN,NaN
2,4366387.0,1,2018,90,144.0,0,0,21846.0,18924.0,21841.0,...,18924.0,21841.0,2442616.0,144.0,144.0,81326.0,81326.0,NaN,NaN,NaN
3,4366406.0,4,2018,90,60.0,0,0,18924.0,18924.0,21841.0,...,18922.0,18922.0,2460526.0,60.0,60.0,181349.0,NaN,NaN,NaN,NaN
4,4366428.0,2,2018,90,47.0,0,0,21841.0,18922.0,21841.0,...,21841.0,18924.0,2446428.0,47.0,47.0,182252.0,182252.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4370750.0,2,2018,4,9996.0,0,0,18922.0,18924.0,21841.0,...,18922.0,18922.0,2452538.0,470.0,470.0,181343.0,181957.0,NaN,NaN,NaN
96,4370757.0,6,2018,4,72.0,0,0,18922.0,18922.0,NaN,...,NaN,NaN,2474934.0,72.0,72.0,81324.0,81324.0,81324.0,NaN,NaN
97,4370758.0,2,2018,4,30.0,0,0,18922.0,NaN,NaN,...,NaN,NaN,2447286.0,30.0,30.0,21952.0,21960.0,NaN,NaN,NaN
98,4370760.0,1,2018,4,30.0,0,0,18924.0,21841.0,21841.0,...,21841.0,NaN,2439379.0,30.0,30.0,21846.0,NaN,NaN,NaN,NaN


In [320]:
#dropping all duplicate ID's and USSCIDN's

import re

booleans = []
ids = []

for result in matches_100.ID:
    if result in ids:
        booleans.append(False)
    if result not in ids:
        booleans.append(True)
        ids.append(result)

matches_100['Counts']=booleans

Filtered = pd.Series(booleans)
matches_100 = matches_100[Filtered]

matches_100 = matches_100.drop_duplicates(subset= ["USSCIDN"], keep='first')

matches_100

,ID,SENTMON,SENTYR,DISTRICT,PRISTOT,FINETOT,PROBTOT,FJDcode1,FJDcode2,FJDcode3,...,FJDcode10,USSCIDN,floorSENTTOT,roundSENTTOT,USSCcode1,USSCcode2,USSCcode3,USSCcode4,USSCcode5,Counts
0,4366217.0,6,2018,90,216.0,0,0,18924.0,18924.0,18924.0,...,18922.0,2472159.0,216.0,216.0,182252.0,NaN,NaN,NaN,NaN,True
1,4366322.0,7,2018,90,60.0,0,0,18922.0,NaN,NaN,...,NaN,2478292.0,60.0,60.0,NaN,NaN,NaN,NaN,NaN,True
2,4366387.0,1,2018,90,144.0,0,0,21846.0,18924.0,21841.0,...,21841.0,2442616.0,144.0,144.0,81326.0,81326.0,NaN,NaN,NaN,True
3,4366406.0,4,2018,90,60.0,0,0,18924.0,18924.0,21841.0,...,18922.0,2460526.0,60.0,60.0,181349.0,NaN,NaN,NaN,NaN,True
4,4366428.0,2,2018,90,47.0,0,0,21841.0,18922.0,21841.0,...,18924.0,2446428.0,47.0,47.0,182252.0,182252.0,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4370750.0,2,2018,4,9996.0,0,0,18922.0,18924.0,21841.0,...,18922.0,2452538.0,470.0,470.0,181343.0,181957.0,NaN,NaN,NaN,True
96,4370757.0,6,2018,4,72.0,0,0,18922.0,18922.0,NaN,...,NaN,2474934.0,72.0,72.0,81324.0,81324.0,81324.0,NaN,NaN,True
97,4370758.0,2,2018,4,30.0,0,0,18922.0,NaN,NaN,...,NaN,2447286.0,30.0,30.0,21952.0,21960.0,NaN,NaN,NaN,True
98,4370760.0,1,2018,4,30.0,0,0,18924.0,21841.0,21841.0,...,NaN,2439379.0,30.0,30.0,21846.0,NaN,NaN,NaN,NaN,True


In [322]:
matches_25=matches_100.iloc[0:25,:] #subsetting to 25 cases

In [323]:
#extracting ID's from cases we know match
fjc_vals= matches_25.ID
ussc_vals=matches_25.USSCIDN

In [324]:
#putting together ID's that we know match
matches_25_df = matches_25[['ID', 'USSCIDN' ]]

In [325]:
#creating ussc subset data
ussc_5 = pd.merge(matches_25_df, ussc_new, on=['USSCIDN'])
ussc_5=ussc_5.drop(['ID'], axis=1)
ussc_5


,USSCIDN,SENTMON,SENTYR,DISTRICT,PRISTOT,FINETOT,PROBTOT,floorSENTTOT,roundSENTTOT,USSCcode1,USSCcode2,USSCcode3,USSCcode4,USSCcode5
0,2472159.0,6,2018,90,216.0,0,0,216.0,216.0,182252.0,NaN,NaN,NaN,NaN
1,2478292.0,7,2018,90,60.0,0,0,60.0,60.0,NaN,NaN,NaN,NaN,NaN
2,2442616.0,1,2018,90,144.0,0,0,144.0,144.0,81326.0,81326.0,NaN,NaN,NaN
3,2460526.0,4,2018,90,60.0,0,0,60.0,60.0,181349.0,NaN,NaN,NaN,NaN
4,2446428.0,2,2018,90,47.0,0,0,47.0,47.0,182252.0,182252.0,NaN,NaN,NaN
5,2489957.0,9,2018,90,80.0,0,0,80.0,80.0,NaN,NaN,NaN,NaN,NaN
6,2489580.0,9,2018,90,60.0,0,0,60.0,60.0,NaN,NaN,NaN,NaN,NaN
7,2479700.0,7,2018,90,240.0,0,0,240.0,240.0,NaN,NaN,NaN,NaN,NaN
8,2447060.0,2,2018,90,46.0,0,0,46.0,46.0,18922.0,18922.0,18924.0,NaN,NaN
9,2485592.0,8,2018,90,36.0,0,0,36.0,36.0,NaN,NaN,NaN,NaN,NaN


In [326]:
#creating fjc subset data
fjc_5 = pd.merge(matches_25_df, fjc_new, on=['ID'])
fjc_5=fjc_5.drop(['USSCIDN'], axis=1)
fjc_5


,ID,SENTMON,SENTYR,DISTRICT,PRISTOT,FINETOT,PROBTOT,FJDcode1,FJDcode2,FJDcode3,FJDcode4,FJDcode5,FJDcode6,FJDcode7,FJDcode8,FJDcode9,FJDcode10
0,4366217.0,6,2018,90,216.0,0,0,18924,18924.0,18924.0,21841.0,18922.0,21841,18924.0,18924.0,21841.0,18922.0
1,4366322.0,7,2018,90,60.0,0,0,18922,NaN,NaN,NaN,NaN,18922,NaN,NaN,NaN,NaN
2,4366387.0,1,2018,90,144.0,0,0,21846,18924.0,21841.0,21841.0,18922.0,21846,18924.0,21846.0,18924.0,21841.0
3,4366406.0,4,2018,90,60.0,0,0,18924,18924.0,21841.0,18922.0,18922.0,18924,18924.0,21841.0,18922.0,18922.0
4,4366428.0,2,2018,90,47.0,0,0,21841,18922.0,21841.0,18924.0,18924.0,18922,21841.0,18922.0,21841.0,18924.0
5,4366433.0,9,2018,90,80.0,0,0,18924,18924.0,21841.0,18922.0,21841.0,18924,21841.0,18924.0,18922.0,21841.0
6,4366485.0,9,2018,90,60.0,0,0,18924,21841.0,21841.0,18922.0,21841.0,18922,21841.0,18924.0,21841.0,21841.0
7,4366504.0,7,2018,90,240.0,0,0,18924,21841.0,18922.0,NaN,NaN,21841,18922.0,18924.0,NaN,NaN
8,4366538.0,2,2018,90,46.0,0,0,18922,NaN,NaN,NaN,NaN,18922,NaN,NaN,NaN,NaN
9,4366557.0,8,2018,90,36.0,0,0,18924,21841.0,18922.0,18922.0,21844.0,18922,21844.0,18924.0,21841.0,18922.0


In [327]:
#resetting indices and subsetting
ussc_5.reset_index(drop=True, inplace=True)
fjc_5.reset_index(drop=True, inplace=True)
ussc_5 = ussc_5.iloc[0:10,:]
fjc_5 = fjc_5.iloc[0:10,:]

In [328]:
ussc_5

,USSCIDN,SENTMON,SENTYR,DISTRICT,PRISTOT,FINETOT,PROBTOT,floorSENTTOT,roundSENTTOT,USSCcode1,USSCcode2,USSCcode3,USSCcode4,USSCcode5
0,2472159.0,6,2018,90,216.0,0,0,216.0,216.0,182252.0,NaN,NaN,NaN,NaN
1,2478292.0,7,2018,90,60.0,0,0,60.0,60.0,NaN,NaN,NaN,NaN,NaN
2,2442616.0,1,2018,90,144.0,0,0,144.0,144.0,81326.0,81326.0,NaN,NaN,NaN
3,2460526.0,4,2018,90,60.0,0,0,60.0,60.0,181349.0,NaN,NaN,NaN,NaN
4,2446428.0,2,2018,90,47.0,0,0,47.0,47.0,182252.0,182252.0,NaN,NaN,NaN
5,2489957.0,9,2018,90,80.0,0,0,80.0,80.0,NaN,NaN,NaN,NaN,NaN
6,2489580.0,9,2018,90,60.0,0,0,60.0,60.0,NaN,NaN,NaN,NaN,NaN
7,2479700.0,7,2018,90,240.0,0,0,240.0,240.0,NaN,NaN,NaN,NaN,NaN
8,2447060.0,2,2018,90,46.0,0,0,46.0,46.0,18922.0,18922.0,18924.0,NaN,NaN
9,2485592.0,8,2018,90,36.0,0,0,36.0,36.0,NaN,NaN,NaN,NaN,NaN


In [329]:
ussc_5.iloc[0,2]

2018

In [330]:
fjc_5

,ID,SENTMON,SENTYR,DISTRICT,PRISTOT,FINETOT,PROBTOT,FJDcode1,FJDcode2,FJDcode3,FJDcode4,FJDcode5,FJDcode6,FJDcode7,FJDcode8,FJDcode9,FJDcode10
0,4366217.0,6,2018,90,216.0,0,0,18924,18924.0,18924.0,21841.0,18922.0,21841,18924.0,18924.0,21841.0,18922.0
1,4366322.0,7,2018,90,60.0,0,0,18922,NaN,NaN,NaN,NaN,18922,NaN,NaN,NaN,NaN
2,4366387.0,1,2018,90,144.0,0,0,21846,18924.0,21841.0,21841.0,18922.0,21846,18924.0,21846.0,18924.0,21841.0
3,4366406.0,4,2018,90,60.0,0,0,18924,18924.0,21841.0,18922.0,18922.0,18924,18924.0,21841.0,18922.0,18922.0
4,4366428.0,2,2018,90,47.0,0,0,21841,18922.0,21841.0,18924.0,18924.0,18922,21841.0,18922.0,21841.0,18924.0
5,4366433.0,9,2018,90,80.0,0,0,18924,18924.0,21841.0,18922.0,21841.0,18924,21841.0,18924.0,18922.0,21841.0
6,4366485.0,9,2018,90,60.0,0,0,18924,21841.0,21841.0,18922.0,21841.0,18922,21841.0,18924.0,21841.0,21841.0
7,4366504.0,7,2018,90,240.0,0,0,18924,21841.0,18922.0,NaN,NaN,21841,18922.0,18924.0,NaN,NaN
8,4366538.0,2,2018,90,46.0,0,0,18922,NaN,NaN,NaN,NaN,18922,NaN,NaN,NaN,NaN
9,4366557.0,8,2018,90,36.0,0,0,18924,21841.0,18922.0,18922.0,21844.0,18922,21844.0,18924.0,21841.0,18922.0


In [332]:
#generating cost matrix
#each variable that isn't a match increases cost by 1
#not considering US statute codes yet

cost_matrix = np.zeros((10,10))

for i in range (10):
    for j in range(10):
        value=0
        if ussc_5.iloc[i,1] != fjc_5.iloc[j,1]:
            value=value+1
        if ussc_5.iloc[i,2] != fjc_5.iloc[j,2]:
            value=value+1
        if ussc_5.iloc[i,3] != fjc_5.iloc[j,3]:
            value=value+1
        if ussc_5.iloc[i,4] != fjc_5.iloc[j,4]:
            value=value+1
        if ussc_5.iloc[i,5] != fjc_5.iloc[j,5]:
            value=value+1
        if ussc_5.iloc[i,6] != fjc_5.iloc[j,6]:
            value=value+1
        cost_matrix[i][j] = value
        
cost_matrix

array([[0., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
       [2., 0., 2., 1., 2., 2., 1., 1., 2., 2.],
       [2., 2., 0., 2., 2., 2., 2., 2., 2., 2.],
       [2., 1., 2., 0., 2., 2., 1., 2., 2., 2.],
       [2., 2., 2., 2., 0., 2., 2., 2., 1., 2.],
       [2., 2., 2., 2., 2., 0., 1., 2., 2., 2.],
       [2., 1., 2., 1., 2., 1., 0., 2., 2., 2.],
       [2., 1., 2., 2., 2., 2., 2., 0., 2., 2.],
       [2., 2., 2., 2., 1., 2., 2., 2., 0., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2., 2., 0.]])

In [333]:
#weighing all the variables equally
#S is our optimal tranport plan!
#the cases along the diagonal should be optimal matching
a=[1,1,1,1,1,1,1,1,1,1]
b=[1,1,1,1,1,1,1,1,1,1]
M= cost_matrix

S = ot.emd(a,b,M)
S

/var/folders/rp/lgskb8cx1vzgzb8nb18p5vxw0000gn/T/ipykernel_43359/1239581406.py:8: UserWarning: Input histogram consists of integer. The transport plan will be casted accordingly, possibly resulting in a loss of precision. If this behaviour is unwanted, please make sure your input histogram consists of floating point elements.
  S = ot.emd(a,b,M)


array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])